In [1]:
import joblib
EXP_PATH_NAME="SCNN-OCSVM"
RIVAL_PATH_NAME="WACA-OCSVM"
joblib.cpu_count()

32

In [2]:
# !pip install --upgrade pip

import os
from joblib import Parallel, delayed

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dataclasses
from sklearn.svm import OneClassSVM
from dataclasses import asdict
from tqdm import tqdm
import warnings
import random
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold # Feature selector

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns


# Global utitlity functions are in separate notebook
%run ./Classification_utility-functions.ipynb
%run ./SEED-CONSTANTS.ipynb

WINDOW_SIZE_LST=WINDOW_SIZE_LST
np.random.seed(SEED)
print(f"Numpy Seed was set to: {SEED}")

print("Setup Complete")

SEED: 567
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
EER: 0.333, Threshold: 0.600 <-- Arbitrary case
EER: 0.000, Threshold: 0.900 <-- Best case
EER: 1.000, Threshold: 0.900 <-- Worse case
EER: 0.400, Threshold: 0.200 <-- Worse case
EER: 0.167, Threshold: 0.600 <-- Arbitrary case
EER: 0.000, Threshold: 0.900 <-- Best case
EER: 1.000, Threshold: 0.900 <-- Worse case
EER: 0.333, Threshold: 1.000 <-- Worse case
--------------------Utility functions imported--------------------
SEED: 567
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
--------------------Preprocessing utility functions imported--------------------
SEED: 567
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
--------------------Neural Networks utility functions imported--------------------
SEED: 567
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
EER

In [3]:
import sklearn
sklearn.__dir__()
sklearn.__version__

'1.0.2'

In [4]:
@dataclasses.dataclass
class ExperimentParameters:
    """Contains all relevant parameters to run an experiment."""

    name: str  # Name of Parameter set. Used as identifier for charts etc.
    frequency: int
    max_subjects: int
    max_test_subjects: int
        
    user_ids: list
    num_sample_points_per_exp: int
    exp_begin_cutoff_idx: int
    exp_end_cutoff_idx: int
        
    
    seconds_per_subject_train: float
    seconds_per_subject_test: float
    window_size: int  # After resampling
    ocsvm_step_width: int  # After resampling
    scaler: str  # StandardScaler, MinMaxScaler, Normalizer, MaxAbsScaler, RobustScaler, PowerTransformer
    scaler_scope: str  # {"subject", "session"}
    scaler_global: bool  # fit transform scale on all data (True) or fit on training only (False)
    ocsvm_kernel: str # ocsvm kernel
    ocsvm_nu: float  # Best value found in random search, used for final model
    ocsvm_gamma: float  # Best value found in random search, used for final model
    feature_cols: list  # Columns used as features
    exclude_subjects: list  # Don't load data from those users
        
    # Calculated values
    def __post_init__(self):
        # HDF key of table:
        self.table_name = f"sensors_{self.frequency}hz"

        

# INSTANCES
# ===========================================================

# NAIVE_APPROACH
# -----------------------------------------------------------
NAIVE_MINMAX_OCSVM = ExperimentParameters(
    name="NAIVE-MINMAX_OCSVM",
    frequency=100,
    max_subjects=29,
    max_test_subjects=10,
    user_ids = [1, 2, 3, 4, 5, 6, 7, 8, 19, 21, 22, 26, 27, 28, 29, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49],
    num_sample_points_per_exp=21000,
    exp_begin_cutoff_idx=500,
    exp_end_cutoff_idx=-500,
    seconds_per_subject_train=210,
    seconds_per_subject_test=210,    
    window_size=250,
    ocsvm_step_width=250,
    scaler="minmax",
    scaler_scope="subject",
    scaler_global=True,
    ocsvm_kernel="rbf",
    ocsvm_nu=None,
    ocsvm_gamma=None,
    feature_cols=[
        "x_a",
        "y_a",
        "z_a",
        "x_g",
        "y_g",
        "z_g",
    ],
    exclude_subjects=[],
)

# VALID_APPROACH
# -----------------------------------------------------------
VALID_MINMAX_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-MINMAX-OCSVM",
    scaler_global=False,
    ocsvm_nu=0.165,
    ocsvm_gamma=0.039,
)

# NAIVE_ROBUST_APPROACH
# -----------------------------------------------------------
NAIVE_ROBUST_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="NAIVE-ROBUST-OCSVM",
    scaler="robust",
    scaler_global=True,
    ocsvm_nu=0.153,
    ocsvm_gamma=0.091,  # below median, selected by chart
)

# ROBUST_APPROACH (VALID)
# -----------------------------------------------------------
VALID_ROBUST_OCSVM_125 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=125
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)


VALID_ROBUST_OCSVM_250 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=250
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_500 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=500
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_750 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=750
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1000 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1000
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1250 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1250
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1500 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1500
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1750 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1750
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_2000 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=2000
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

# NORMALIZER_APPROACH (VALID)
# -----------------------------------------------------------
VALID_NORMALIZER_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-NORMALIZER-OCSVM",
    scaler="Normalizer",
    scaler_global=False,
    ocsvm_nu=0.074,
    ocsvm_gamma= 0.029,
)

In [5]:
P = VALID_ROBUST_OCSVM_2000
P.ocsvm_step_width = int(P.window_size * .5)
P.classifier="OCSVM"
P.ocsvm_kernel = "rbf"
P.train_cores=1 # 20 cores for every user and 1 core for the nested crossval function
P.test_cores=2 # 10 cores for every user and 2 for the nested crossval function


test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{RIVAL_PATH_NAME}-df.json"
RIVAL_DF = pd.read_json(test_file_name)
RIVAL_DF
# Convert the string representation of tuples back to tuples
idx_tuples = [eval(i) for i in RIVAL_DF.columns.tolist()]

# # Convert list of tuples back to a MultiIndex
multiindex_columns = pd.MultiIndex.from_tuples(idx_tuples)

RIVAL_DF.columns = multiindex_columns
RIVAL_DF

Window Size            \
  mean rank cut_off_freq EMA_span SMA_winsize   type         125       250   
0         2           41        -           -  Naive    0.227313  0.171836   
1         3           49        -           -   Real    0.226914  0.170020   
2         5            -       25           -  Naive    0.192994  0.158703   
3         9            -        9           -   Real    0.362315  0.309108   
4         6            -        -          41  Naive    0.196284  0.157013   
5        10            -        -           1   Real    0.397188  0.357325   
6         1           37       10           -  Naive    0.213821  0.164012   
7         7           16        2           -   Real    0.291194  0.220732   
8         4           39        -          41  Naive    0.225110  0.174325   
9         8           20        -           4   Real    0.318503  0.235023   

                                                                         \
        500       750      1000      1250      1500      1750      2000   
0  0.141258  0.139333  0.131003  0.125799  0.115267  0.111014  0.093000   
1  0.146292  0.141475  0.127317  0.127153  0.117160  0.112222  0.090389   
2  0.142343  0.153697  0.140488  0.136146  0.155267  0.136812  0.151167   
3  0.259384  0.226525  0.220000  0.193125  0.189506  0.173382  0.171278   
4  0.149103  0.146646  0.149404  0.142569  0.149465  0.153333  0.152778   
5  0.298996  0.253192  0.242033  0.221389  0.213580  0.182802  0.184500   
6  0.146345  0.141535  0.134824  0.117049  0.110988  0.107729  0.114778   
7  0.175596  0.164606  0.154146  0.157569  0.128642  0.115894  0.117333   
8  0.153025  0.147010  0.138320  0.139410  0.127531  0.128937  0.132056   
9  0.188969  0.172444  0.161192  0.154618  0.130700  0.123575  0.134222   

                       
       mean  variance  
0  0.139536  0.001573  
1  0.139882  0.001572  
2  0.151957  0.000306  
3  0.233847  0.004283  
4  0.155177  0.000255  
5  0.261223  0.005684  
6  0.139009  0.001142  
7  0.169524  0.003142  
8  0.151747  0.000968  
9  0.179916  0.003882

In [6]:
P.smoothing = "Butterworth"


#range(1000, 2001, 250)#
preprocessing_method="Naive"
rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)
rival_test_hyperparameters_df


,window_size,step_width,Mean_EER,median_nu,median_gamma,cut_off_freq
0,125,62,0.227313,0.035779,0.655129,41
1,250,125,0.171836,0.055147,0.372759,41
2,500,250,0.141258,0.041895,0.372759,41
3,750,375,0.139333,0.034759,0.120679,41
4,1000,500,0.131003,0.035779,0.068665,41
5,1250,625,0.125799,0.040876,0.068665,41
6,1500,750,0.115267,0.033740,0.068665,41
7,1750,875,0.111014,0.031701,0.068665,41
8,2000,1000,0.093000,0.034759,0.068665,41


In [7]:
utils_ppp(P)

,Value
name,VALID-ROBUST-OCSVM
frequency,100
max_subjects,29
max_test_subjects,10
user_ids,"[1, 2, 3, 4, 5, 6, 7, 8, 19, 21, 22, 26, 27, 2..."
num_sample_points_per_exp,21000
exp_begin_cutoff_idx,500
exp_end_cutoff_idx,-500
seconds_per_subject_train,210
seconds_per_subject_test,210


In [8]:
utils_eer_scorer

make_scorer(utils_eer, greater_is_better=False)

In [9]:
#include 47 later
# user_ids = [9]
df_exps_dict = load_data_frames(P.user_ids, P.exp_begin_cutoff_idx, P.exp_end_cutoff_idx, P.num_sample_points_per_exp)
raw_dfList_exp1, raw_dfList_exp2 = df_exps_dict['dfList_exp1'], df_exps_dict['dfList_exp2']

Loading exp1 data:
1) accel_count: 28388, gyro_count: 31997
2) accel_count: 26010, gyro_count: 28954
3) accel_count: 28227, gyro_count: 31814
4) accel_count: 24860, gyro_count: 26105
5) accel_count: 24270, gyro_count: 24347
6) accel_count: 25012, gyro_count: 25060
7) accel_count: 25301, gyro_count: 25382
8) accel_count: 21975, gyro_count: 21658
19) accel_count: 24110, gyro_count: 25050
21) accel_count: 24326, gyro_count: 23809
22) accel_count: 29123, gyro_count: 28724
26) accel_count: 23148, gyro_count: 24291
27) accel_count: 24299, gyro_count: 23589
28) accel_count: 23807, gyro_count: 24523
29) accel_count: 24030, gyro_count: 23457
35) accel_count: 24388, gyro_count: 23673
36) accel_count: 24228, gyro_count: 24208
37) accel_count: 31945, gyro_count: 31816
38) accel_count: 22135, gyro_count: 22327
39) accel_count: 23573, gyro_count: 23459
40) accel_count: 23057, gyro_count: 24296
41) accel_count: 24102, gyro_count: 23681
42) accel_count: 24074, gyro_count: 24328
43) accel_count: 22631,

In [10]:
randomized_data_idx = list(range(len(P.user_ids)))
random.Random(SEED).shuffle(randomized_data_idx)
split_idx = 2 * (len(randomized_data_idx)//3) + 1
train_set = randomized_data_idx[: split_idx]
test_set = randomized_data_idx[split_idx: ]
# train_set = randomized_data_idx
print(f"train_set: {train_set}\ntest_set: {test_set}")
# train_set = test_set
# test_set = train_set
print(f"train_set: {train_set}\ntest_set: {test_set}")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]


In [11]:
num_sample_points_per_exp_user_47 = 18000
df_exps_dict_user_47 = load_data_frames([47], P.exp_begin_cutoff_idx, P.exp_end_cutoff_idx, num_sample_points_per_exp_user_47)
dfList_exp1_user_47, dfList_exp2_user_47 = df_exps_dict_user_47['dfList_exp1'], df_exps_dict_user_47['dfList_exp2']

raw_dfList_exp1_user_47 = dfList_exp1_user_47
raw_dfList_exp2_user_47 = dfList_exp2_user_47

Loading exp1 data:
47) accel_count: 22777, gyro_count: 22226
Loading exp2 data:
47) accel_count: 17718, gyro_count: 18353


In [12]:
print(f"train_set: {train_set}")
# print(f"X_exp1_train_dic: {X_exp1_train_dic.keys()}")
# print(f"X_exp2_train_dic: {X_exp2_train_dic.keys()}")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5]


In [13]:
print(f"test_set: {test_set}")
# print(f"X_exp1_test_dic: {X_exp1_test_dic.keys()}")
# print(f"X_exp2_test_dic: {X_exp2_test_dic.keys()}")

test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]


In [14]:
WINDOW_SIZE_LST

[125, 250, 500, 750, 1000, 1250, 1500, 1750, 2000]

In [15]:
int('121')
SAVE_PATH_NAME
# loss_func = lambda y_true, dist: k_contrastive_loss(y_true, dist, arg_dict["contrastive_loss_margin"])
model_classifier_type

NameError: name 'model_classifier_type' is not defined

In [50]:
extract_learnt_features

<function __main__.extract_learnt_features(X_exp1_dic, X_exp2_dic, fitted_scaler_classifier_exp2_dic, deep_feature_model, verbose=1)>

In [26]:
def get_exp_deep_feature_extractors_dict(exp_num, model_architectures= list(TRAINING_CONFIG_DICT['2000'].keys())):
    exp_deep_feature_extractors_dict={}

    model_classifier_type=EXP_PATH_NAME
    base_path=f"final_experiments_results/{SAVE_PATH_NAME}/{model_classifier_type}/{exp_num}"
    for archi_name in model_architectures:
        # archi_name="SCNN_3_1_conv_1_dense_arg_dict_default"
        path_to_archi_name=base_path+f"/{archi_name}"
        archi_win_dict={}
        for winsize in WINDOW_SIZE_LST:
            final_path=path_to_archi_name+f"/{winsize}"
            deep_feature_extractor_path=final_path+f"/{model_classifier_type}-{exp_num}-{archi_name}-{winsize}-deep_feature_extractor"
            print(deep_feature_extractor_path)
            
            deep_feature_model=load_model(deep_feature_extractor_path)#, custom_objects={'loss_func': loss_func})
            
            deep_feature_extractor = partial_func(extract_learnt_features, deep_feature_model=deep_feature_model)

            archi_win_dict[winsize]=deep_feature_extractor

        exp_deep_feature_extractors_dict[archi_name]=archi_win_dict
        
    return exp_deep_feature_extractors_dict

# exp_deep_feature_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_1_1)

In [17]:
# Create a function to suppress TensorFlow warnings
def suppress_tf_warnings():
    warnings.filterwarnings('ignore', category=UserWarning)
    tf.get_logger().setLevel('ERROR')

# Call the function before loading the model
suppress_tf_warnings()

def enable_tf_warnings():
    warnings.filterwarnings('default', category=UserWarning)
    tf.get_logger().setLevel('WARN')

# Call the function to enable warnings
# enable_tf_warnings()

In [17]:
TESTING_CONFIG_DICT.items()

NameError: name 'TESTING_CONFIG_DICT' is not defined

In [64]:

X_train_exp1_dict, X_train_exp2_dict, fitted_scaler_SNN_train_exp2_dict=get_raw_windows(raw_dfList_exp1, raw_dfList_exp2, 1750, 1750//2, train_set, 
                                                                    P.scaler, P.num_sample_points_per_exp, EMA_per_win_span=None, SMA_per_win_winsize=None)


len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19


In [56]:
X_train_exp1_dict[7][0].shape

(2000, 6)

In [89]:
TESTING_CONFIG_DICT["SCNN_1_3_conv_1_dense_arg_dict_default"]

{'SCNN_1_3_conv_1_dense_arg_dict_default': {...},
 'SCNN_3_123_conv_1_dense_arg_dict_default': {...},
 'SCNN_3_1_conv_1_dense_arg_dict_default': {...},
 'SCNN_1_2_conv_1_dense_arg_dict_default': {...},
 'SCNN_1_1_conv_1_dense_arg_dict_default': {...},
 'SCNN_1_2_conv_2_dense_arg_dict_default': {...},
 'SCNN_1_5_conv_1_dense_arg_dict_default': {...},
 'SCNN_4_1234_conv_1_dense_arg_dict_default': {...}}

In [94]:
a=np.array([X_train_exp1_dict[7][0]])

In [23]:
# for model_name, model in TESTING_CONFIG_DICT.items():
#     print(model[1750].predict(a))

In [24]:
# archi_name="SCNN_1_2_conv_1_dense_arg_dict_default"
# base_path=f"final_experiments_results/{SAVE_PATH_NAME}/{model_classifier_type}/{EXP_NUM_1_1}"
# path_to_archi_name=base_path+f"/{archi_name}"
# final_path=path_to_archi_name+f"/{winsize_str}"
# deep_feature_extractor_path=final_path+f"/{model_classifier_type}-{EXP_NUM_1_1}-{archi_name}-{winsize_str}-deep_feature_extractor"
# model=load_model(deep_feature_extractor_path)

In [25]:
model.summary()

NameError: name 'model' is not defined

In [ ]:
base_path=f"final_experiments_results/{SAVE_PATH_NAME}/{EXP_PATH_NAME}/{EXP_NUM_1_1}"
path_to_archi_name=base_path+f"/{archi_name}"
final_path=path_to_archi_name+f"/{winsize}"
    
    if not os.path.isdir(base_path):
        raise Exception(f"Base path does not exist: {base_path}")
        
    if not os.path.isdir(path_to_archi_name):
        os.mkdir(path_to_archi_name)

    if not os.path.isdir(final_path):
        os.mkdir(final_path)
        
    model.save(final_path+f"/{model_classifier_type}-{exp_num}-{archi_name}-{winsize}-model")
    deep_feature_model = extract_deep_feature_extactor(model)
    deep_feature_model.save(final_path+f"/{model_classifier_type}-{exp_num}-{archi_name}-{winsize}-deep_feature_extractor")

In [25]:

# def correct_archi_extractor_dict_key_names(exp_deep_archi_extractors_dict):
#     items=list(exp_deep_archi_extractors_dict.items())
#     for key, val in items:
#         exp_deep_archi_extractors_dict[key[:-len("_arg_dict_default")]]=val
#         # del exp_deep_archi_extractors_dict[key]
#     return exp_deep_archi_extractors_dict
# exp_deep_archi_extractors_dict=correct_archi_extractor_dict_key_names(exp_deep_feature_extractors_dict)
# exp_deep_archi_extractors_dict.keys()

In [28]:
exp_deep_archi_extractors_dict.keys()
WINDOW_SIZE_LST = [125, 250, 500, 750, 1000, 1250, 1500, 1750, 2000]

In [ ]:
enable_tf_warnings()


In [30]:
RIVAL_PATH_NAME

'WACA-OCSVM'

In [36]:
rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
print(f"cut_off_freq: {P.cut_off_freq}")
rival_test_hyperparameters_df

cut_off_freq: -99999


,window_size,step_width,Mean_EER,median_nu,median_gamma,cut_off_freq
0,125,62,0.231879,0.042914,0.655129,-99999
1,250,125,0.178423,0.040876,0.655129,-99999
2,500,250,0.152316,0.040876,0.372759,-99999
3,750,375,0.146990,0.035779,0.120679,-99999
4,1000,500,0.133523,0.034759,0.120679,-99999
5,1250,625,0.128229,0.034759,0.068665,-99999
6,1500,750,0.137243,0.034759,0.030650,-99999
7,1750,875,0.108889,0.035779,0.068665,-99999
8,2000,1000,0.106611,0.035779,0.039069,-99999


In [29]:
get_exp_deep_feature_extractors_dict(EXP_NUM_0, model_architectures= [list(TRAINING_CONFIG_DICT['2000'].keys())[0]])


final_experiments_results/SCNN_trained_models/SCNN-OCSVM/0/SCNN_1_3_conv_1_dense_arg_dict_default/125/SCNN-OCSVM-0-SCNN_1_3_conv_1_dense_arg_dict_default-125-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/0/SCNN_1_3_conv_1_dense_arg_dict_default/250/SCNN-OCSVM-0-SCNN_1_3_conv_1_dense_arg_dict_default-250-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/0/SCNN_1_3_conv_1_dense_arg_dict_default/500/SCNN-OCSVM-0-SCNN_1_3_conv_1_dense_arg_dict_default-500-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/0/SCNN_1_3_conv_1_dense_arg_dict_default/750/SCNN-OCSVM-0-SCNN_1_3_conv_1_dense_arg_dict_default-750-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/0/SCNN_1_3_conv_1_dense_arg_dict_default/1000/SCNN-OCSVM-0-SCNN_1_3_conv_1_dense_arg_dict_default-1000-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/0/SCNN_1_3_conv_1_dense_arg_dict_def

{'SCNN_1_3_conv_1_dense_arg_dict_default': {125: functools.partial(<function extract_learnt_features at 0x7f41c94283a0>, deep_feature_model=<keras.engine.functional.Functional object at 0x7f2b577db520>),
  250: functools.partial(<function extract_learnt_features at 0x7f41c94283a0>, deep_feature_model=<keras.engine.functional.Functional object at 0x7f2b4f609f10>),
  500: functools.partial(<function extract_learnt_features at 0x7f41c94283a0>, deep_feature_model=<keras.engine.functional.Functional object at 0x7f2b468dc1f0>),
  750: functools.partial(<function extract_learnt_features at 0x7f41c94283a0>, deep_feature_model=<keras.engine.functional.Functional object at 0x7f2b4a9fad60>),
  1000: functools.partial(<function extract_learnt_features at 0x7f41c94283a0>, deep_feature_model=<keras.engine.functional.Functional object at 0x7f2b4a9fd6d0>),
  1250: functools.partial(<function extract_learnt_features at 0x7f41c94283a0>, deep_feature_model=<keras.engine.functional.Functional object at 0x

# 0. No Smoothing
### Optimizing and Testing

In [39]:
init_experiment_params(exp_config=P)

# exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_0)
exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(EXP_NUM_0, model_architectures= [list(TRAINING_CONFIG_DICT['2000'].keys())[0]])

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = None

preprocessing_method=None
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")


EER_df_test_dict={}
for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():

    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    print(f"cut_off_freq: {P.cut_off_freq}")

    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": raw_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": raw_dfList_exp2_user_47
    }
    
    test_dict_key=scnn_archi_name
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, 
                                                                                                          test_set=test_set, exp_config=P, 
                                                                                                          extract_features_func_dict=exp_deep_feature_extractors_dict, 
                                                                                                          overlap=OVERLAP, 
                                                                                                          best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"test_dict_key: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
        
    
#-------
#-------
key_column= ["scnn_archi_name"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')

reseting experiment params successful!
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/0/SCNN_1_3_conv_1_dense_arg_dict_default/125/SCNN-OCSVM-0-SCNN_1_3_conv_1_dense_arg_dict_default-125-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/0/SCNN_1_3_conv_1_dense_arg_dict_default/250/SCNN-OCSVM-0-SCNN_1_3_conv_1_dense_arg_dict_default-250-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/0/SCNN_1_3_conv_1_dense_arg_dict_default/500/SCNN-OCSVM-0-SCNN_1_3_conv_1_dense_arg_dict_default-500-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/0/SCNN_1_3_conv_1_dense_arg_dict_default/750/SCNN-OCSVM-0-SCNN_1_3_conv_1_dense_arg_dict_default-750-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/0/SCNN_1_3_conv_1_dense_arg_dict_default/1000/SCNN-OCSVM-0-SCNN_1_3_conv_1_dense_arg_dict_default-1000-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCS

  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  1.3784460476599634
MakeSCNNXExpDicUnknown Time:  36.01882671285421
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 3487.99it/s]

 20%|██        | 1/5 [00:07<00:30,  7.61s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7665.03it/s]

 40%|████      | 2/5 [00:08<00:11,  3.72s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 9218.25it/s]

 60%|██████    | 3/5 [00:09<00:04,  2.49s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8504.27it/s]

 80%|████████  | 4/5 [00:10<00:01,  1.76s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 10713.42it/s]

 11%|█         | 1/9 [00:49<06:33, 49.15s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  1.41061616782099
MakeSCNNXExpDicUnknown Time:  21.557730227708817
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9628.80it/s]

 20%|██        | 1/5 [00:00<00:00,  4.93it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 10519.95it/s]

 40%|████      | 2/5 [00:00<00:00,  5.09it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 10594.35it/s]

 60%|██████    | 3/5 [00:00<00:00,  4.99it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 10143.42it/s]

 80%|████████  | 4/5 [00:00<00:00,  5.01it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 6395.71it/s]

 78%|███████▊  | 7/9 [02:22<00:30, 15.11s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  0.882440104149282
MakeSCNNXExpDicUnknown Time:  10.21187306707725
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9279.43it/s]

 20%|██        | 1/5 [00:00<00:00,  4.84it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 11599.29it/s]

 40%|████      | 2/5 [00:00<00:00,  5.27it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 6638.66it/s]

 60%|██████    | 3/5 [00:00<00:00,  5.05it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 14378.83it/s]

 80%|████████  | 4/5 [00:00<00:00,  5.20it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 7851.56it/s]

 89%|████████▉ | 8/9 [02:35<00:14, 14.20s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  0.9358865288086236
MakeSCNNXExpDicUnknown Time:  10.004068272188306
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10300.35it/s]

 20%|██        | 1/5 [00:00<00:00,  4.73it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8505.99it/s]

 40%|████      | 2/5 [00:00<00:00,  4.87it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 6859.04it/s]

 60%|██████    | 3/5 [00:00<00:00,  5.06it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8103.37it/s]

 80%|████████  | 4/5 [00:00<00:00,  5.04it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 9345.60it/s]

100%|██████████| 9/9 [02:47<00:00, 18.58s/it]


In [4]:
# exp_deep_archi_extractors_dict.items()

NameError: name 'exp_deep_archi_extractors_dict' is not defined

# 1. Butterworth frequency Cut-off

## 1.1 Naive Approach
### Optimizing and Testing

In [41]:
init_experiment_params(exp_config=P)

# exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_1_1)
exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(EXP_NUM_1_1, model_architectures= [list(TRAINING_CONFIG_DICT['2000'].keys())[0]])

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butterworth"


preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    



overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}

for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():

    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    print(f"cut_off_freq: {P.cut_off_freq}")

    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)


    dfList_dict={
                "dfList_exp1": ffted_dfList_exp1,
                "dfList_exp2": ffted_dfList_exp2,
                "dfList_exp1_user_47": ffted_dfList_exp1_user_47,
                "dfList_exp2_user_47": ffted_dfList_exp2_user_47
    }



    min_key=rival_test_hyperparameters_df["cut_off_freq"][0]
    test_dict_key=min_key, scnn_archi_name
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, 
                                                                                                          test_set=test_set, exp_config=P, 
                                                                                                          extract_features_func_dict=exp_deep_feature_extractors_dict, 
                                                                                                          overlap=OVERLAP, 
                                                                                                          best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"cut_off_freq: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
        
    
#-------
#-------
key_column= ["cut_off_freq", "scnn_archi_name"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')

reseting experiment params successful!
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.1/SCNN_1_3_conv_1_dense_arg_dict_default/125/SCNN-OCSVM-1.1-SCNN_1_3_conv_1_dense_arg_dict_default-125-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.1/SCNN_1_3_conv_1_dense_arg_dict_default/250/SCNN-OCSVM-1.1-SCNN_1_3_conv_1_dense_arg_dict_default-250-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.1/SCNN_1_3_conv_1_dense_arg_dict_default/500/SCNN-OCSVM-1.1-SCNN_1_3_conv_1_dense_arg_dict_default-500-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.1/SCNN_1_3_conv_1_dense_arg_dict_default/750/SCNN-OCSVM-1.1-SCNN_1_3_conv_1_dense_arg_dict_default-750-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.1/SCNN_1_3_conv_1_dense_arg_dict_default/1000/SCNN-OCSVM-1.1-SCNN_1_3_conv_1_dense_arg_dict_default-1000-deep_feature_extractor
final_experiments_results/SCNN_tra

  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  1.3732085549272597
MakeSCNNXExpDicUnknown Time:  36.0077731590718
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 7898.88it/s]

 20%|██        | 1/5 [00:07<00:30,  7.55s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8481.91it/s]

 40%|████      | 2/5 [00:08<00:11,  3.67s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8391.96it/s]

 60%|██████    | 3/5 [00:09<00:04,  2.44s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7949.78it/s]

 80%|████████  | 4/5 [00:10<00:01,  1.76s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 7850.09it/s]

 11%|█         | 1/9 [00:49<06:32, 49.03s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  1.1335460939444602
MakeSCNNXExpDicUnknown Time:  21.330917451065034
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 8876.83it/s]

 20%|██        | 1/5 [00:00<00:00,  4.91it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 6709.81it/s]

 40%|████      | 2/5 [00:00<00:00,  5.02it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 9018.07it/s]

 60%|██████    | 3/5 [00:00<00:00,  5.08it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8217.68it/s]

 80%|████████  | 4/5 [00:00<00:00,  5.15it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 5919.98it/s]

 89%|████████▉ | 8/9 [02:36<00:14, 14.21s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  0.9188729957677424
MakeSCNNXExpDicUnknown Time:  10.006054670084268
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 8845.01it/s]

 20%|██        | 1/5 [00:00<00:00,  5.07it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 10580.99it/s]

 40%|████      | 2/5 [00:00<00:00,  5.19it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 14222.80it/s]

 60%|██████    | 3/5 [00:00<00:00,  5.23it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 10200.16it/s]

 80%|████████  | 4/5 [00:00<00:00,  5.12it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 9825.03it/s]

100%|██████████| 9/9 [02:48<00:00, 18.72s/it]

Columns must be same length as key
the number of columns in key_column must match the number of elements of dict key!
len(key_column): 1 != len(key): 2
type(key_column): <class 'list'>, type(key): <class 'tuple'>


AttributeError: 'NoneType' object has no attribute 'copy'

In [5]:
key_column= test_dict_key#["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')

NameError: name 'test_dict_key' is not defined

In [18]:
exp_deep_archi_extractors_dict.keys()

dict_keys(['SCNN_1_3_conv_1_dense_arg_dict_default', 'SCNN_3_123_conv_1_dense_arg_dict_default', 'SCNN_3_1_conv_1_dense_arg_dict_default', 'SCNN_1_2_conv_1_dense_arg_dict_default', 'SCNN_1_1_conv_1_dense_arg_dict_default', 'SCNN_1_2_conv_2_dense_arg_dict_default', 'SCNN_1_5_conv_1_dense_arg_dict_default', 'SCNN_4_1234_conv_1_dense_arg_dict_default'])

ERROR! Session/line number was not unique in database. History logging moved to new session 1227


In [19]:
K.clear_session()
tf.compat.v1.reset_default_graph()

In [ ]:
window_size_lst=WINDOW_SIZE_LST
exp_config=P
extract_features_func_dict=exp_deep_archi_extractors_dict["SCNN_1_3_conv_1_dense_arg_dict_default"]
dfList_exp1=dfList_dict["dfList_exp1"]
dfList_exp2=dfList_dict["dfList_exp2"]
EER_distro_df_dict = {}

if exp_config.classifier=="OCSVM":
    columns = ['window_size', "step_width", "Mean_EER", "median_nu", "median_gamma"]

elif exp_config.classifier=="IF":
    columns = ['window_size', "step_width", "Mean_EER", "median_n_estimators", "median_max_samples", "median_contamination", "median_max_features"]

elif exp_config.classifier=="LOF":
    columns = ['window_size', "step_width", "Mean_EER", "median_n_neighbors", "median_contamination"]

else:
    raise Exception("Unspecified classifier")

Mean_EER_df = pd.DataFrame(columns = columns)
for window_size in tqdm(window_size_lst):
    print(f"window_size: {window_size}")
    step_width = int(window_size * (1-overlap))


    X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict=get_raw_windows(dfList_exp1, dfList_exp2, window_size, step_width, test_set, exp_config.scaler, 
                                                                                     exp_config.num_sample_points_per_exp, EMA_per_win_span=exp_config.EMA_per_win_span, 
                                                                                     SMA_per_win_winsize=exp_config.SMA_per_win_winsize,
                                                                                     Butter_per_win_argdict=exp_config.Butter_per_win_argdict, 
                                                                                     verbose=0)


    extract_features_func=extract_features_func_dict[window_size]

    X_exp_test_dic = extract_features_func(X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict)
    
X_exp_test_dic


  0%|          | 0/2 [00:00<?, ?it/s]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10


  0%|          | 0/2 [00:02<?, ?it/s]


In [42]:
X_test_exp1_dict[0][0].shape

(1750, 6)

In [50]:
X_test_exp1_dict.keys()

dict_keys([0, 12, 14, 9, 18, 23, 2, 15, 10, 4])

In [38]:
#X_test_exp1_dict

In [28]:
base_path=f"final_experiments_results/{SAVE_PATH_NAME}/SCNN-OCSVM/{EXP_NUM_1_1}"
archi_name = list(TRAINING_CONFIG_DICT['2000'].keys())[0]
# archi_name="SCNN_3_1_conv_1_dense_arg_dict_default"
path_to_archi_name=base_path+f"/{archi_name}"
archi_win_dict={}
final_path=path_to_archi_name+f"/{1750}"
final_path

'final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.1/SCNN_1_3_conv_1_dense_arg_dict_default/1750'

In [35]:
loss_func = lambda y_true, dist: k_contrastive_loss(y_true, dist, arg_dict["contrastive_loss_margin"])

In [46]:
deep_feature_extractor_path=final_path+f"/SCNN-OCSVM-{EXP_NUM_1_1}-{archi_name}-{1750}-deep_feature_extractor"
print(deep_feature_extractor_path)

deep_feature_model=load_model(deep_feature_extractor_path)# custom_objects={'loss_func': loss_func})

final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.1/SCNN_1_3_conv_1_dense_arg_dict_default/1750/SCNN-OCSVM-1.1-SCNN_1_3_conv_1_dense_arg_dict_default-1750-deep_feature_extractor
ERROR! Session/line number was not unique in database. History logging moved to new session 1233


In [47]:
deep_feature_model.predict(np.array([X_test_exp1_dict[0][0]]))

array([[0.483646  , 0.48248574, 0.48391485, 0.5176617 , 0.48232555,
        0.3948905 , 0.6571675 , 0.48332578, 0.51537776, 0.3830801 ,
        0.48369867, 0.51706755, 0.6734431 , 0.48458332, 0.48752847,
        0.48411527, 0.40387475, 0.518302  , 0.5168323 , 0.48319304,
        0.5119249 , 0.36883786, 0.51656306, 0.48535657, 0.5160692 ,
        0.49645782, 0.6151099 , 0.3447324 , 0.5164715 , 0.5170238 ,
        0.48302537, 0.50929   , 0.48378727, 0.48427698, 0.37200788,
        0.4824282 , 0.5118454 , 0.5513184 , 0.5161465 , 0.52022976,
        0.51684374, 0.48289943, 0.5146176 , 0.3277162 , 0.5170986 ,
        0.51055324, 0.49970394, 0.48305914, 0.4838405 , 0.48391843,
        0.5158296 , 0.48297504, 0.48362806, 0.48273656, 0.5167648 ,
        0.4758899 , 0.5162947 , 0.48710927, 0.5125867 , 0.51675326,
        0.65629625, 0.5182427 , 0.6739007 , 0.665079  , 0.48348317,
        0.4852233 , 0.48333368, 0.3298231 , 0.48392308, 0.33389822,
        0.5162636 , 0.5160936 , 0.5179101 , 0.48

In [48]:
ExtractDeepFeatures([X_test_exp1_dict], deep_feature_model)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type dict).

In [ ]:
extract_learnt_features(X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict, deep_feature_model, verbose=1)

In [ ]:
MakeDeepXExpDic(X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict, 
                    X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict, deep_feature_model)


In [ ]:
extract_features_func(X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict)

ERROR! Session/line number was not unique in database. History logging moved to new session 1234


## 1.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

exp_deep_feature_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_1_2)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butterworth"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    






P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }

overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}

for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():
    P.cut_off_freq=cut_off_freq
    P.Butter_per_win_argdict["cut_off_freq"]=cut_off_freq
    print(f"cut_off_freq: {P.cut_off_freq}")

    
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": ffted_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": ffted_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.cut_off_freq] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                                    extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                                    param_dist=param_dist)
        
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq: {P.cut_off_freq}\n")
        f.write(EER_df_train_dict[P.cut_off_freq].to_string())


mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# 2. Butterworth frequency Cut-off + EMA span
## 2.1 Naive Approach
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

exp_deep_feature_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_2_1)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_span_pairs: {index_pairs.shape}, choice_num: {choice_num}")
cut_off_span_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]







overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}

for key_pair in tqdm(cut_off_span_pairs):
    
    key_pair = tuple(key_pair)
    cut_off_freq, span = key_pair[0], key_pair[1]
    P.cut_off_freq=cut_off_freq
    P.span=span
    print(f"cut_off_freq: {P.cut_off_freq}")
    print(f"span: {P.span}")

    
    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    EMAed_dfList_exp1 = get_EMAed_dfList(ffted_dfList_exp1, span=P.span)
    EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)
    
    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    EMAed_dfList_exp1_user_47 = get_EMAed_dfList(ffted_dfList_exp1_user_47, span=P.span)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)
    
    dfList_dict={
                "dfList_exp1": EMAed_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                      param_dist=param_dist)
        
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, span: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------  
#-------
key_column= ["cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")
        

## 2.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

exp_deep_feature_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_2_2)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_span_pairs: {index_pairs.shape}, choice_num: {choice_num}")
cut_off_span_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]






P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }

overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}

for key_pair in tqdm(cut_off_span_pairs):
    key_pair = tuple(key_pair)
    cut_off_freq, span = key_pair[0], key_pair[1]
    P.cut_off_freq=cut_off_freq
    P.Butter_per_win_argdict["cut_off_freq"]=cut_off_freq
    
    
    P.span=span
    P.EMA_per_win_span=span

    print(f"cut_off_freq: {P.cut_off_freq}")
    print(f"span: {P.span}")

    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, span: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# 3. EMA span
## 3.1 Naive Approach
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

exp_deep_feature_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_3_1)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    
    



overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}
for span in tqdm(list(range(1, 50, 23))):
    P.span=span
    print(f"EMA span: {P.span}")

    
    EMAed_dfList_exp1 = get_EMAed_dfList(raw_dfList_exp1, span=P.span)
    EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)
    
    EMAed_dfList_exp1_user_47 = get_EMAed_dfList(raw_dfList_exp1_user_47, span=P.span)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)

    
    dfList_dict={
                "dfList_exp1": EMAed_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.span] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                    extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                    param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\EMA span: {P.span}\n")
        f.write(EER_df_train_dict[P.span].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

## 3.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

exp_deep_feature_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_3_2)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    
 


overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}
for span in tqdm(list(range(1, 50, 23))):
    P.span=span
    print(f"EMA span: {P.span}")

    P.EMA_per_win_span=P.span

    EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)
    
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.span] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                    extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                    param_dist=param_dist)
        
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\EMA span: {P.span}\n")
        f.write(EER_df_train_dict[P.span].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# 4. SMA winsize
## 4.1 Naive Approach
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

exp_deep_feature_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_4_1)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    
    



overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}
for winsize in tqdm(list(range(1, 50, 23))):
    P.winsize=winsize
    print(f"SMA winsize: {P.winsize}")


    SMAed_dfList_exp1 = get_SMAed_dfList(raw_dfList_exp1, winsize=P.winsize)
    SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)
    
    SMAed_dfList_exp1_user_47 = get_SMAed_dfList(raw_dfList_exp1_user_47, winsize=P.winsize)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)

    
    dfList_dict={
                "dfList_exp1": SMAed_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.winsize] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                       extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                       param_dist=param_dist)
        
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\SMA winsize: {P.winsize}\n")
        f.write(EER_df_train_dict[P.winsize].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

## 4.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

exp_deep_feature_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_4_2)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    
    



overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}
for winsize in tqdm(list(range(1, 50, 23))):
    P.winsize=winsize
    P.SMA_per_win_winsize=P.winsize

    print(f"SMA winsize: {P.winsize}")


    SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)
    
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.winsize] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                       extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                       param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\SMA winsize: {P.winsize}\n")
        f.write(EER_df_train_dict[P.winsize].to_string())

        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# 5. Butterworth frequency Cut-off + SMA winsize
## 5.1 Naive Approach
### Optimizing and Testin

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

exp_deep_feature_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_5_1)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_winsize_pairs: {index_pairs.shape}, choice_num: {choice_num}")
cut_off_winsize_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]






overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}

for key_pair in tqdm(cut_off_winsize_pairs):
    
    key_pair = tuple(key_pair)
    cut_off_freq, winsize = key_pair[0], key_pair[1]
    P.cut_off_freq=cut_off_freq
    P.winsize=winsize
    print(f"cut_off_freq: {P.cut_off_freq}")
    print(f"winsize: {P.winsize}")

    
    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    SMAed_dfList_exp1 = get_SMAed_dfList(ffted_dfList_exp1, winsize=P.winsize)
    SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)
    
    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    SMAed_dfList_exp1_user_47 = get_SMAed_dfList(ffted_dfList_exp1_user_47, winsize=P.winsize)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)
    
    dfList_dict={
                "dfList_exp1": SMAed_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                      param_dist=param_dist)

    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, winsize: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

In [ ]:
tuple(key_pair)

## 5.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

exp_deep_feature_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_5_2)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_winsize_pairs: {index_pairs.shape}, choice_num: {choice_num}")
cut_off_winsize_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]






P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }

overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}

for key_pair in tqdm(cut_off_winsize_pairs):
    
    key_pair = tuple(key_pair)
    cut_off_freq, winsize = key_pair[0], key_pair[1]
    P.cut_off_freq=cut_off_freq
    P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
    P.winsize=winsize
    P.SMA_per_win_winsize=P.winsize
    
    print(f"cut_off_freq: {P.cut_off_freq}")
    print(f"winsize: {P.winsize}")

    
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, winsize: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())

        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# 6. The effect of Varying Overlap

In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")



time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/overlap_Mean_EER_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/overlap_Mean_EER_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    


EER_df_train_dict={}
EER_df_test_dict={}
for overlap in tqdm(list(range(50, 52))):
    overlap*=0.01
    max_window_size=2000
    step_width = int(max_window_size * (1-overlap))
    max_num_windows=min(len(getIndices(sampleSize=max_window_size, step=step_width, numSamplePoints=P.num_sample_points_per_exp)), 20)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": raw_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": raw_dfList_exp2_user_47
    }
    

    EER_df_train_dict[overlap] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                     extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                     param_dist=param_dist)
        
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\overlap: {overlap}\n")
        f.write(EER_df_train_dict[overlap].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=extract_WACA_features, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["overlap"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

In [ ]:
1

# Summary

In [ ]:

df=return_and_save_final_result_df_as_json(final_exp_results_path=FINAL_EXP_RESULTS_PATH, exp_path_name=EXP_PATH_NAME, window_size_lst=WINDOW_SIZE_LST)
df

In [ ]:
relative_df=return_and_save_final_relative_result_df_as_json(df, base_case_index=0, final_exp_results_path=FINAL_EXP_RESULTS_PATH, exp_path_name=EXP_PATH_NAME, 
                                                             window_size_lst=WINDOW_SIZE_LST)
relative_df

In [ ]:
df.style.hide(axis='index').to_latex()


In [ ]:
relative_df.style.hide(axis='index').to_latex()
